In [28]:
from manage_geo import * 
from config import config
import os
import sys

In [3]:
gse_gpl = []
with open("/pstore/data/bioinfo/users/sturmg/BioQC_GEO_analysis/gse_tissue_annot/gse_gpl.tsv") as f: 
    for i, l in enumerate(f.readlines()):
        if i == 0: 
            continue
        gse_gpl.append([x.strip('"') for x in l.split()[:2]])

## Flatfiles

In [ ]:
!rm -f /copy_from_BioQC_GEO_analysis.log
sys.stderr = open('./copy_from_BioQC_GEO_analysis.log', 'w')

In [4]:
path = "/pstore/data/bioinfo/users/sturmg/BioQC_GEO_analysis/gse_tissue_annot/geo_annot_flat"
suffixes = [('exprs', 'gct'), ('fdata', 'tsv'), ('pdata', 'tsv')]

In [5]:
gse_dict = {
    gse: [] for gse, _ in gse_gpl
}

In [6]:
for gse, gpl in gse_gpl:
    gse_dict[gse].append(gpl)
    assert len(set(gse_dict[gse])) == len(gse_dict[gse])

In [10]:
for gse in gse_dict:
    if(len(gse_dict[gse]) == 1):
        base_filenames = [os.path.join(path, gse + '_{}.{}')]
    else: 
        base_filenames = [os.path.join(path, '{}-{}'.format(gse, gpl) + '_{}.{}') for gpl in gse_dict[gse]]

    for base_filename, gpl in zip(base_filenames, gse_dict[gse]):
        for sfx, ext in suffixes: 
            file = os.path.join(path, base_filename.format(sfx, ext))
            if(os.path.exists(file)): 
                put({'FILE': file, 'GSE': gse, 'GPL': gpl, 'SUFFIX': sfx, '--move': False, '--force': False})
            sys.stderr.flush()

## Rdata objects

In [14]:
!rm -f /copy_from_BioQC_GEO_analysis.rdata.log
sys.stderr = open('./copy_from_BioQC_GEO_analysis.rdata.log', 'w')

In [15]:
path = '/pstore/data/bioinfo/users/sturmg/BioQC_GEO_analysis/gse_tissue_annot/geo_annot'

In [16]:
for gse in gse_dict:
    if(len(gse_dict[gse]) == 1):
        base_filenames = [os.path.join(path, gse + '.Rdata')]
    else: 
        base_filenames = [os.path.join(path, '{}-{}'.format(gse, gpl) + '.Rdata') for gpl in gse_dict[gse]]

    for base_filename, gpl in zip(base_filenames, gse_dict[gse]):
        file = os.path.join(path, base_filename)
        if(os.path.exists(file)): 
            put({'FILE': file, 'GSE': gse, 'GPL': gpl, 'SUFFIX': 'eset', '--move': False, '--force': False})
        sys.stderr.flush()

## Make list
make a nice list with with the BioQC-GEO dataset (~81k samples, 2900 studies)

In [27]:
bioqc_geo_list = [] 
with open("/pstore/home/sturmg/projects/BioQC_GEO_analysis/results/gse_lists/bioqced_esets.txt") as f:
    for l in f.readlines():
        ident = l.split('_')[0]
        if '-' in ident: 
            gse, gpl = ident.split('-')
            if not gpl in gse_dict[gse]:
                print('invalid gpl {}, {}'.format(gse, gpl))
                continue
        else: 
            gse = ident
            if not len(gse_dict[gse]) == 1:
                print('identifier not unique, {}, {}, {}'.format(gse, gse_dict[gse], l))
                continue
            gpl = gse_dict[gse][0]

        bioqc_geo_list.append('{}-{}'.format(gse, gpl))

identifier not unique, GSE33564, ['GPL14860', 'GPL6247', 'GPL10906'], GSE33564_bioqc_res_melt.tab

invalid gpl GSE35321, GPL6887
identifier not unique, GSE59787, ['GPL4133', 'GPL6480'], GSE59787_bioqc_res_melt.tab

invalid gpl GSE64770, GPL6885
invalid gpl GSE73037, GPL6480
identifier not unique, GSE75285, ['GPL570', 'GPL16298', 'GPL6801'], GSE75285_bioqc_res_melt.tab



In [29]:
comment = """# List of studies which 
# * have probe-ids mapped to human orthologous gene symbols
# * have a tissue of origin annotated
# * can be successfully processed with BioQC"""

In [31]:
list_file = os.path.join(config['root_dir'], 'index', 'GSE_tissue_and_gene_symbols.txt')

In [32]:
with open(list_file, 'w') as f: 
    f.write(comment + '\n')
    for ident in bioqc_geo_list:
        f.write(ident + '\n')

In [34]:
!wc -l {list_file}

7807 /pstore/data/bi/reference/GEO/index/GSE_tissue_and_gene_symbols.txt
